<a href="https://colab.research.google.com/github/ngupta23/ds7337_nlp/blob/master/Homework/NGupta_homework8_with_sigmoid.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Homework 8.
Utilize the Universal Sentence encoder to build a model for the following dataset.  Note--we want to POLARITY not Sentiment column, although the data is setup correctly already



In [1]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import os
import re

In [2]:
import numpy as np

In [3]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.io.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)

  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))

  return train_df, test_df

# Reduce logging output.
#tf.logging.set_verbosity(tf.logging.ERROR)

train_df, test_df = download_and_load_datasets()
train_df.head()

,sentence,sentiment,polarity
0,**May Contain Spoilers**<br /><br />A dude in ...,3,0
1,This 1981 comedy still sparkles thanks to the ...,7,1
2,There are so many stupid moments in 'Tower of ...,4,0
3,What starts out as generational conflict in th...,7,1
4,"Had this movie been made a few years later, I ...",8,1


In [4]:
# Create datasets (Only take up to 150 words for memory)
train_text = train_df['sentence'].tolist()
train_text = [' '.join(t.split()[0:150]) for t in train_text]
train_text = np.array(train_text, dtype=object)[:, np.newaxis]
train_label = train_df['polarity']

test_text = test_df['sentence'].tolist()
test_text = [' '.join(t.split()[0:150]) for t in test_text]
test_text = np.array(test_text, dtype=object)[:, np.newaxis]
test_label = test_df['polarity']

In [5]:
print(len(train_text), len(train_label))
print(len(test_text), len(test_label))

25000 25000
25000 25000


In [6]:
train_text[0].shape

(1,)

In [7]:
set(train_label)

{0, 1}

In [8]:
use_test = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
use_test(["This is a test of USE"])

<tf.Tensor: shape=(1, 512), dtype=float32, numpy=
array([[-8.41595232e-03, -5.61952703e-02, -4.07758318e-02,
        -2.94545945e-02, -4.88133729e-02, -7.73191154e-02,
        -1.81259564e-03,  3.17418166e-02,  4.34551425e-02,
         4.00216877e-03,  1.36102820e-02, -1.15882112e-02,
         6.16076216e-02, -2.87124584e-03,  3.92416082e-02,
         2.57035792e-02,  2.41553877e-03,  3.72053608e-02,
         1.08773699e-02, -5.01606390e-02,  3.74448746e-02,
        -2.02413765e-03, -5.03091924e-02, -1.17291445e-02,
        -4.35480587e-02,  1.77046638e-02,  2.77202819e-02,
        -6.90487027e-02,  2.61135884e-02, -8.04910958e-02,
         4.13932502e-02, -8.80538151e-02, -8.11734819e-04,
        -1.49555169e-02, -8.64357222e-03,  8.20124075e-02,
        -8.62774497e-04, -8.40719640e-02,  2.60526314e-02,
         5.91145493e-02, -2.58402415e-02, -4.03626189e-02,
        -2.41041593e-02, -3.43766361e-02, -1.90319698e-02,
        -2.09738836e-02, -3.37156504e-02,  2.57028062e-02,
      

In [9]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout

In [10]:
use = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",trainable=False)

"""
OOPS I DELETED ALL MY CODE!!  I guess you have to build a network that will do sentiment analysis with 
the Universal Sentance Encoder (USE) from above!

Note that USE layer can be utilized with Sequential() or Model()
"""

# This is useful to debug the issues with input shape: https://github.com/tensorflow/hub/issues/526
input_text = Input(shape=[], name='sent1',dtype=tf.string)
embedding = use(input_text)
# dense1 = Dense(256, activation='relu')(embedding)
# dropout1 = Dropout(0.2)(dense1)
# dense2 = Dense(100, activation='relu')(dropout1)
dense2 = Dense(100, activation='relu')(embedding)
dropout2 = Dropout(0.2)(dense2)
# pred = Dense(2, activation='softmax')(dropout2)
pred = Dense(1, activation='sigmoid')(dropout2)
model = Model(inputs=[input_text], outputs=pred)

model.summary()
tf.keras.utils.plot_model(model, show_shapes=True, dpi=100)

loss = tf.keras.losses.BinaryCrossentropy()
# loss = tf.keras.losses.SparseCategoricalCrossentropy()
model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])

# TODO: Questions
# Previously, we had only 1 output that was needed. https://github.com/ngupta23/ds7337_nlp/blob/master/Homework/NGupta_homework3_withstopwords_recdropout.ipynb
# For this one, it seems that we need 2. Why is this? 
# Does this have to do with Sequential vs. non Sequential model?
# But that can not be the case, since here we used non sequential but still had 1 output https://github.com/ngupta23/ds7337_nlp/blob/master/Homework/NGupta_homework7_max_seq_500_rmsprop.ipynb

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sent1 (InputLayer)           [(None,)]                 0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 512)               256797824 
_________________________________________________________________
dense (Dense)                (None, 100)               51300     
_________________________________________________________________
dropout (Dropout)            (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,849,225
Trainable params: 51,401
Non-trainable params: 256,797,824
_________________________________________________________________


In [11]:
model.predict(tf.convert_to_tensor(["the cat is on the mat", "dogs are in the fog"]))

array([[0.49041137],
       [0.4745564 ]], dtype=float32)

In [12]:
from tensorflow.keras.callbacks import EarlyStopping
# If validation error does not improve for 10 epochs, then terminate the training
es = EarlyStopping(min_delta=0,patience=3)

from tensorflow.keras.callbacks import ModelCheckpoint
# add a checkpoint to save the lowest validation loss
filepath = '.'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True, save_weights_only=False, 
                             mode='auto', save_frequency=1)

In [13]:
BATCH_SIZE = 64
EPOCHS = 10
history = model.fit(
    train_text, train_label,
    validation_data=(test_text, test_label),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[es, checkpoint]    
    )

Epoch 1/10
391/391 [==============================] - ETA: 0s - loss: 0.4380 - accuracy: 0.8064
Epoch 00001: val_loss improved from inf to 0.38594, saving model to .
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: ./assets


...

391/391 [==============================] - 26s 67ms/step - loss: 0.3520 - accuracy: 0.8445 - val_loss: 0.3720 - val_accuracy: 0.8282
Epoch 8/10
391/391 [==============================] - ETA: 0s - loss: 0.3459 - accuracy: 0.8465
Epoch 00008: val_loss did not improve from 0.37204
391/391 [==============================] - 19s 47ms/step - loss: 0.3459 - accuracy: 0.8465 - val_loss: 0.3721 - val_accuracy: 0.8291
Epoch 9/10
390/391 [============================>.] - ETA: 0s - loss: 0.3404 - accuracy: 0.8526
Epoch 00009: val_loss improved from 0.37204 to 0.37102, saving model to .
INFO:tensorflow:Assets written to: ./assets


INFO:tensorflow:Assets written to: ./assets


391/391 [==============================] - 26s 67ms/step - loss: 0.3404 - accuracy: 0.8526 - val_loss: 0.3710 - val_accuracy: 0.8314
Epoch 10/10
390/391 [============================>.] - ETA: 0s - loss: 0.3357 - accuracy: 0.8529
Epoch 00010: val_loss improved from 0.37102 to 0.36969, saving model to .
INFO:tensorflow:Assets written to: ./assets


INFO:tensorflow:Assets written to: ./assets


391/391 [==============================] - 26s 66ms/step - loss: 0.3357 - accuracy: 0.8528 - val_loss: 0.3697 - val_accuracy: 0.8318


In [20]:
model.predict(tf.convert_to_tensor(["the cat is on the mat", "dogs are in the fog"]))

array([[0.40208486],
       [0.9054954 ]], dtype=float32)

In [21]:
predictions = model.predict(test_text)
predictions[:5]

array([[0.94386214],
       [0.4842739 ],
       [0.4215925 ],
       [0.08084611],
       [0.5498696 ]], dtype=float32)

In [25]:
predictions_max = (predictions > 0.5).astype(int)
predictions_max[:5]

array([[1],
       [0],
       [0],
       [0],
       [1]])

In [26]:
test_label[0:5]

0    1
1    1
2    0
3    1
4    1
Name: polarity, dtype: int64

In [27]:
m = tf.keras.metrics.Accuracy()
m.update_state(test_label, predictions_max)
m.result().numpy()

0.8318

In [28]:
predictions

array([[0.94386214],
       [0.4842739 ],
       [0.4215925 ],
       ...,
       [0.7230363 ],
       [0.86783254],
       [0.30410177]], dtype=float32)